## Herança múltipla e super

Quando usamos herança, ao redefinir um método na classe derivada é comum que queiramos usar o mesmo método da classe base. Isso ocorre quase sempre no caso do método de inicialização `__init__`.

Infelizmente, quando existe herança múltipla, existe um problema, pois mais de uma classe base podem ser derivadas da mesma classe base original, e nesse caso sua inicialização seria executada duas vezes.

Veja no exemplo do código abaixo, onde a classe `D` é derivada da classe `A` por dois caminhos: por `B` e por `C`.

In [1]:
class A:
    def __init__(self):
        print('+A')
class B(A):
    def __init__(self):
        print('+B')
        A.__init__(self)
class C(A):
    def __init__(self):
        print('+C')
        A.__init__(self)
class D(B, C):
    def __init__(self):
        print('+D')
        B.__init__(self)
        C.__init__(self)

O resultado é que, ao criar um objeto da classe `D`, `A.__init__` será chamado duas vezes:

In [2]:
d = D()

+D
+B
+A
+C
+A


Uma forma de resolver esse problema (e também simplificar um pouco a notação), é usar a função `super`, que permite chamar um método conforme definido na classe base, mesmo que ele seja redefinido na classe derivada. Quando usamos `super` o Python automaticamente toma conta da herança múltipla e de garantir que cada método seja chamado apenas uma vez.

In [3]:
class AN:
    def __init__(self):
        print('+A')
class BN(AN):
    def __init__(self):
        print('+B')
        super().__init__()
class CN(AN):
    def __init__(self):
        print('+C')
        super().__init__()
class DN(BN, CN):
    def __init__(self):
        print('+D')
        super().__init__()

Com essa definição, agora `A.__init__` será chamado apenas uma vez.

In [4]:
dn = DN()

+D
+B
+C
+A


No entanto, `super` não é totalmente sem problemas, pois ele força uma ordem específica na execução dos métodos das classes base, que pode não ser a mais adequada em alguns casos.

## Exceções

Já vimos que, quando detectamos um erro ou outra situação excepcional que impedem a continuidade da execução normal do código, podemos lançar uma exceção usando o comando `raise`.

In [5]:
raise Exception('Erro!')
print('Não passo aqui')

Exception: Erro!

Como sabemos, se nada for feito a exceção irá interromper a execução do código com uma mensagem de erro, como acima, que não executa o `print`.

Mas existe a possibilidade de detectar que a exceção ocorreu e lidar com ela, sem interromper o programa. Para isso, colocamos o código que pode gerar exceção dentro de um bloco `try`, e seguimos esse bloco `try` com um (ou mais) tratadores de exceção, como abaixo.

In [6]:
try:
    print('Faz algo.')
    raise Exception('Erro!')
except Exception:
    print('Não deu certo.')
print('Passo por aqui')

Faz algo.
Não deu certo.
Passo por aqui


Normalmente, o `try` envolve uma quantidade maior de código, onde o lançamento da exceção e apenas uma das possibilidades.

O código abaixo tenta re-executar um *loop* se sua execução foi interrompida por exceção.

In [7]:
terminou = False
primeiro = True
while not terminou:
    try:
        for i in range(10):
            if primeiro and i == 4:
                raise Exception('Já cansei.')
            print(i, end=' ')
        print()
        terminou = True
    except Exception:
        print('Não terminou')
        primeiro = False
    print('Agora estou depois do try')
print('Agora terminei')

0 1 2 3 Não terminou
Agora estou depois do try
0 1 2 3 4 5 6 7 8 9 
Agora estou depois do try
Agora terminei


Após o bloco `try`, além de blocos para tratar de exeções, podemos também definir um bloco `finally`. Esse bloco deve incluir código que será executado tanto se o `try` terminar normalmente (sem exeções), como se ele for interrompido por exceção, **mesmo que a exceção não seja capturada por um tratador apropriado**.

No código abaixo, note como `'Fim da função boba'` é impresso tanto no término normal como no termino por interrupção.

In [8]:
def funcao_boba(quebra):
    try:
        if quebra:
            raise ValueError()
        print('Isso')
        for i in range(3):
            print('yeah!', end='')
        print()
    except ValueError: # Experimente retirar este tratador
        print('Quero outro valor')
    finally:
        print('Fim da função boba')

funcao_boba(True)
funcao_boba(False)

Quero outro valor
Fim da função boba
Isso
yeah!yeah!yeah!
Fim da função boba


Um outro tipo de bloco que pode ser adicionado após o `try` é o bloco `else`. O código desse bloco será executado apenas se o `try` **não** for interrompido por uma exceção.

In [9]:
def funcao_boba(quebra):
    try:
        if quebra:
            raise ValueError()
        print('Isso')
        for i in range(3):
            print('yeah!', end='')
        print()
    except ValueError:
        print('Quero outro valor')
    else:
        print('Você foi eficiente')
    finally:
        print('Fim da função boba')

funcao_boba(True)
funcao_boba(False)

Quero outro valor
Fim da função boba
Isso
yeah!yeah!yeah!
Você foi eficiente
Fim da função boba


Exceções são objetos (como tudo em Python), normalmente derivados da classe `Exception`. Existem diversos tipos de exceção definidos na linguagem. [Veja uma lista aqui.](https://docs.python.org/3/library/exceptions.html)

É frequente que desejemos criar novos tipos de exceção, para exprimir de forma mais clara os tipos de erro do nosso programa.

Para isso, basta definir uma classe derivada de `Exception`. A classe, em princípio, não precisa de nenhum conteúdo novo (mas veja a seguir situação onde isso pode ser útil). Ela servirá apenas, através de seu tipo, para expressar um tipo específico de erro. (Execute o código abaixo diversas vezes para ver a variação aleatória.)

In [10]:
class MauHumorado(Exception):
    pass

In [11]:
import datetime
import random

In [15]:
h, m = random.randint(0,23), random.randint(0,59)
despertador_tocou = datetime.time(h, m)
if despertador_tocou < datetime.time(11,0):
    raise MauHumorado('Acordei muito cedo')
print('Bom dia!')

MauHumorado: Acordei muito cedo

São comuns situações onde, além da informação do *tipo* do erro, queremos também alguma informação de contexto que permita determinar melhor a razão do erro.

Neste caso, é aconselhavel criar novas classes de exceção que recebam essa informação adicional.

In [16]:
class DepositoInvalido(Exception):
    def __init__(self, mess, val):
        # mess é a mensagem de exceção, val é o valor inválido
        self.__val = val
        Exception.__init__(self, mess)
    @property
    def valor(self):
        return self.__val
        
class Conta:
    def __init__(self, inicio):
        if inicio < 0:
            raise DepositoInvalido('Valor inicial negativo', inicio)
        self.__saldo = inicio
    def deposito(self, valor):
        if valor < 0:
            raise DepositoInvalido('Deposito negativo', valor)
        self.__saldo += valor

In [17]:
c = Conta(100)

In [18]:
try:
    c.deposito(-10)
except DepositoInvalido as dep:
    print('O valor', dep.valor,'é inválido')

O valor -10 é inválido


In [ ]:
try:
    c2 = Conta(-10)
except DepositoInvalido as dep:
    print(dep.valor, 'não é um depósito inicial válido')

Até aqui, temos apenas um tratador de exceção para o bloco `try`, mas isso não é obrigatório. Podemos colocar um tratador de exceção para cada tipo de exceção que pode ocorrer durante a execução do `try`. Quando ocorre uma exceção, o Python irá verificar um por um os tratadores de exceção na ordem apresentada, buscando um que seja *compatível* com a exceção gerada. Para ser compatível, ele precisa tratar exatamente a classe da exceção ou uma de suas classes base. Por exemplo, um tratador para `Exception` irá cuidar de todos os tipos (normais) de exceção. (Execute o código abaixo diversas vezes.)

In [30]:
try:
    try:
        raise Exception("Oi")
    except ValueError:
        print("Olá")
    if random.random() < 0.3:
        raise ValueError('Valor')
    elif random.random() < 0.3:
        raise TypeError('Tipo')
    else:
        c.deposito(-10)
except DepositoInvalido as dep:
    print('O valor', dep.valor,'é inválido')
except ValueError:
    print('Deu erro no valor')
except Exception:
    print('Deu erro em algum lugar')


Deu erro em algum lugar


Para concluir, a ordem dos blocos após o bloco `try` deve ser:

1. Blocos de tratamento de exceção (do mais específico para o mais geral).
1. Bloco `else`.
1. Bloco `finally`.
